# GipsyX processor
### 1. Conversion, analysis and merging  of rinex files into 30 h arc files
### 2. TropNominals generation with VMF1 model
### 3. IONEX merging
### 4. Tree files generation (for each year and IONEX file)

In [1]:
import glob
import pandas as pd
import numpy as np
from shutil import rmtree
from multiprocessing import Pool
import os, sys, re, subprocess, calendar

In [2]:
#All on options
tree_options = [
                    # Stochastic Adjustment State
                    ['GRN_STATION_CLK_WHITE:State:Pos:StochasticAdj', '10 10 $GLOBAL_DATA_RATE WHITENOISE'],
                    # Tides section
                    ['GRN_STATION_CLK_WHITE:Tides:All', 'On'],
#                     ['GRN_STATION_CLK_WHITE:Tides:OceanLoad', '$OCEANLOAD'], #not needed as "all on"
                    ['GRN_STATION_CLK_WHITE:Tides:OceanLoadFile', '/mnt/Data/bogdanm/tmp_GipsyX/otl/ocnld_coeff/bigf.blq'],
                    # Trop section    
                    ['GRN_STATION_CLK_WHITE:Trop:GradEast', '0.0'],
                    ['GRN_STATION_CLK_WHITE:Trop:GradEast:StochasticAdj', '1.0 5e-6 $GLOBAL_DATA_RATE RANDOMWALK'],
                    ['GRN_STATION_CLK_WHITE:Trop:GradNorth', '0.0'],
                    ['GRN_STATION_CLK_WHITE:Trop:GradNorth:StochasticAdj', '1.0 5e-6 $GLOBAL_DATA_RATE RANDOMWALK'],
                    ['GRN_STATION_CLK_WHITE:Trop:Mapping', 'VMF1'],
                    ['GRN_STATION_CLK_WHITE:Trop:Model', 'On'],
                    ['GRN_STATION_CLK_WHITE:Trop:WetZ', '0.1'],
                    ['GRN_STATION_CLK_WHITE:Trop:WetZ:StochasticAdj', '0.5 5e-5 $GLOBAL_DATA_RATE RANDOMWALK'],
                    # Station properties. e.g. CAMB. So we will get wetz etc in the output. Highly important!!!
                    ['Station', '`cat $STATIONLIST`'+'\nStation `staDb2TreeIn.py -s $STATIONLIST -y2kSecs $GLOBAL_EPOCH -d $STA_DB`'],
                    # VMF1dataDir path  
                    ['Station:VMF1dataDir', '/mnt/Data/bogdanm/Products/VMF1_Products'],
                    # Ion2nd
                    ['Global:Ion2nd','On'],
                    ['Global:Ion2nd:MagneticModel','IGRF'],
                    ['Global:Ion2nd:MagneticModel:IgrfCoefficientsFile', '$GOA_VAR/etc/igrf/igrf11coeffs.txt'],
                    ['Global:Ion2nd:StecModel', 'IONEX'],
                    ['Global:Ion2nd:StecModel:IonexFile:ShellHeight', '600.0e3'],
                    # GPS_BlockII as it is missing from default tree file
                    ['GPS_BlockII_Model', '=='],
                    ['GPS_BlockII_Model:AttitudeModel', 'gpsBlockII'],
                    # Satellite section needed for Gipsy to be able to extract satellite properties. Highly important!!!
                    ['Satellite','`cat $GNSSLIST`'+ '\nSatellite `pcm.py -file $GNSS_ANT_OFF_PCM -epoch $GLOBAL_EPOCH -sat $GNSSLIST -param Antenna1`'],
   
                    ['Global:Input:TimeDepParms:NameFilter:Satellite\.[C]\w*.*\.Clk\.Bias:Degree','0'],
                    ['Global:Input:TimeDepParms:NameFilter:Satellite\.[C]\w*.*\.Clk\.Bias:MaxFormalError','0.4'],
                    ['Global:Input:TimeDepParms:NameFilter:Satellite\.[C]\w*.*\.Clk\.Bias:Strict','On'],
                    ['Global:Input:TimeDepParms:NameFilter:Satellite\.[C]\w*.*\.Clk\.Bias:Strict:MaxDx','1.0e-6'],
                    # Section with namefilter for state
                    ['Global:Input:TimeDepParms:NameFilter:Station\..*\.State\.Pos\..*',' '],
                    ['Global:Input:TimeDepParms:NameFilter:Station\..*\.Trop.*',' '],
    

    
                    ['Global:DataTypes:IonoFreePhaseC1C5:DataLinkSpec_LC_BDS:PostSmoothEdit','2e5 2e4 0.25 0.2 0.1 .05'],    
                    ['Global:DataTypes:IonoFreePhaseC1C5:DataLinkSpec_LC_BDS:SignalPath:Platforms','.* C.*'],
    
                    ['Global:DataTypes:IonoFreePhaseP1P2:DataLinkSpec_LC_GPS:PostSmoothEdit','2e5 2e4 0.125 0.1 0.05 .025'],
    
                    ['Global:DataTypes:IonoFreeRangeC1C5:DataLinkSpec_PC_BDS:PostSmoothEdit','2e5 2e4 25 20 10 5'],    
                    ['Global:DataTypes:IonoFreeRangeC1C5:DataLinkSpec_PC_BDS:SignalPath:Platforms','.* C.*'],     

                    ['Global:DataTypes:IonoFreeRangeP1P2:DataLinkSpec_PC_GPS:PostSmoothEdit','2e5 2e4 12.5 10 5 2.5'],     
                ]

tree_options_tides_off = [
                    # Stochastic Adjustment State
                    ['GRN_STATION_CLK_WHITE:State:Pos:StochasticAdj', '10 10 $GLOBAL_DATA_RATE WHITENOISE'],
                    # Tides section
                    ['GRN_STATION_CLK_WHITE:Tides:All', 'Off'],
#                     ['GRN_STATION_CLK_WHITE:Tides:OceanLoad', '$OCEANLOAD'], #not needed as "all on"
                    ['GRN_STATION_CLK_WHITE:Tides:OceanLoadFile', '/mnt/Data/bogdanm/tmp_GipsyX/otl/ocnld_coeff/bigf.blq'],
                    # Trop section    
                    ['GRN_STATION_CLK_WHITE:Trop:GradEast', '0.0'],
                    ['GRN_STATION_CLK_WHITE:Trop:GradEast:StochasticAdj', '1.0 5e-6 $GLOBAL_DATA_RATE RANDOMWALK'],
                    ['GRN_STATION_CLK_WHITE:Trop:GradNorth', '0.0'],
                    ['GRN_STATION_CLK_WHITE:Trop:GradNorth:StochasticAdj', '1.0 5e-6 $GLOBAL_DATA_RATE RANDOMWALK'],
                    ['GRN_STATION_CLK_WHITE:Trop:Mapping', 'VMF1'],
                    ['GRN_STATION_CLK_WHITE:Trop:Model', 'On'],
                    ['GRN_STATION_CLK_WHITE:Trop:WetZ', '0.1'],
                    ['GRN_STATION_CLK_WHITE:Trop:WetZ:StochasticAdj', '0.5 5e-5 $GLOBAL_DATA_RATE RANDOMWALK'],
                    # Station properties. e.g. CAMB. So we will get wetz etc in the output. Highly important!!!
                    ['Station', '`cat $STATIONLIST`'+'\nStation `staDb2TreeIn.py -s $STATIONLIST -y2kSecs $GLOBAL_EPOCH -d $STA_DB`'],
                    # VMF1dataDir path  
                    ['Station:VMF1dataDir', '/mnt/Data/bogdanm/Products/VMF1_Products'],
                    # Ion2nd
                    ['Global:Ion2nd','On'],
                    ['Global:Ion2nd:MagneticModel','IGRF'],
                    ['Global:Ion2nd:MagneticModel:IgrfCoefficientsFile', '$GOA_VAR/etc/igrf/igrf11coeffs.txt'],
                    ['Global:Ion2nd:StecModel', 'IONEX'],
                    ['Global:Ion2nd:StecModel:IonexFile:ShellHeight', '600.0e3'],
                    # GPS_BlockII as it is missing from default tree file
                    ['GPS_BlockII_Model', '=='],
                    ['GPS_BlockII_Model:AttitudeModel', 'gpsBlockII'],
                    # Satellite section needed for Gipsy to be able to extract satellite properties. Highly important!!!
                    ['Satellite','`cat $GNSSLIST`'+ '\nSatellite `pcm.py -file $GNSS_ANT_OFF_PCM -epoch $GLOBAL_EPOCH -sat $GNSSLIST -param Antenna1`'],
   
                    ['Global:Input:TimeDepParms:NameFilter:Satellite\.[C]\w*.*\.Clk\.Bias:Degree','0'],
                    ['Global:Input:TimeDepParms:NameFilter:Satellite\.[C]\w*.*\.Clk\.Bias:MaxFormalError','0.4'],
                    ['Global:Input:TimeDepParms:NameFilter:Satellite\.[C]\w*.*\.Clk\.Bias:Strict','On'],
                    ['Global:Input:TimeDepParms:NameFilter:Satellite\.[C]\w*.*\.Clk\.Bias:Strict:MaxDx','1.0e-6'],
                    # Section with namefilter for state
                    ['Global:Input:TimeDepParms:NameFilter:Station\..*\.State\.Pos\..*',' '],
                    ['Global:Input:TimeDepParms:NameFilter:Station\..*\.Trop.*',' '],
    

    
                    ['Global:DataTypes:IonoFreePhaseC1C5:DataLinkSpec_LC_BDS:PostSmoothEdit','2e5 2e4 0.25 0.2 0.1 .05'],    
                    ['Global:DataTypes:IonoFreePhaseC1C5:DataLinkSpec_LC_BDS:SignalPath:Platforms','.* C.*'],
    
                    ['Global:DataTypes:IonoFreePhaseP1P2:DataLinkSpec_LC_GPS:PostSmoothEdit','2e5 2e4 0.125 0.1 0.05 .025'],
    
                    ['Global:DataTypes:IonoFreeRangeC1C5:DataLinkSpec_PC_BDS:PostSmoothEdit','2e5 2e4 25 20 10 5'],    
                    ['Global:DataTypes:IonoFreeRangeC1C5:DataLinkSpec_PC_BDS:SignalPath:Platforms','.* C.*'],     

                    ['Global:DataTypes:IonoFreeRangeP1P2:DataLinkSpec_PC_GPS:PostSmoothEdit','2e5 2e4 12.5 10 5 2.5'],     
                ]

#Might need to create tree options with no otl correction enabled

'''tree with ambres off'''
tree_options_ambres_off = [
                    # Stochastic Adjustment State
                    ['GRN_STATION_CLK_WHITE:State:Pos:StochasticAdj', '10 10 $GLOBAL_DATA_RATE WHITENOISE'],
                    # Tides section
                    ['GRN_STATION_CLK_WHITE:Tides:All', 'On'],
#                     ['GRN_STATION_CLK_WHITE:Tides:OceanLoad', '$OCEANLOAD'], #not needed as "all on"
                    ['GRN_STATION_CLK_WHITE:Tides:OceanLoadFile', '/mnt/Data/bogdanm/tmp_GipsyX/otl/ocnld_coeff/bigf.blq'],
                    # Trop section    
                    ['GRN_STATION_CLK_WHITE:Trop:GradEast', '0.0'],
                    ['GRN_STATION_CLK_WHITE:Trop:GradEast:StochasticAdj', '1.0 5e-6 $GLOBAL_DATA_RATE RANDOMWALK'],
                    ['GRN_STATION_CLK_WHITE:Trop:GradNorth', '0.0'],
                    ['GRN_STATION_CLK_WHITE:Trop:GradNorth:StochasticAdj', '1.0 5e-6 $GLOBAL_DATA_RATE RANDOMWALK'],
                    ['GRN_STATION_CLK_WHITE:Trop:Mapping', 'VMF1'],
                    ['GRN_STATION_CLK_WHITE:Trop:Model', 'On'],
                    ['GRN_STATION_CLK_WHITE:Trop:WetZ', '0.1'],
                    ['GRN_STATION_CLK_WHITE:Trop:WetZ:StochasticAdj', '0.5 5e-5 $GLOBAL_DATA_RATE RANDOMWALK'],
                    # Station properties. e.g. CAMB. So we will get wetz etc in the output. Highly important!!!
                    ['Station', '`cat $STATIONLIST`'+'\nStation `staDb2TreeIn.py -s $STATIONLIST -y2kSecs $GLOBAL_EPOCH -d $STA_DB`'],
                    # VMF1dataDir path  
                    ['Station:VMF1dataDir', '/mnt/Data/bogdanm/Products/VMF1_Products'],
                    # Ion2nd
                    ['Global:Ion2nd','On'],
                    ['Global:Ion2nd:MagneticModel','IGRF'],
                    ['Global:Ion2nd:MagneticModel:IgrfCoefficientsFile', '$GOA_VAR/etc/igrf/igrf11coeffs.txt'],
                    ['Global:Ion2nd:StecModel', 'IONEX'],
                    ['Global:Ion2nd:StecModel:IonexFile:ShellHeight', '600.0e3'],
                    # GPS_BlockII as it is missing from default tree file
                    ['GPS_BlockII_Model', '=='],
                    ['GPS_BlockII_Model:AttitudeModel', 'gpsBlockII'],
                    # Satellite section needed for Gipsy to be able to extract satellite properties. Highly important!!!
                    ['Satellite','`cat $GNSSLIST`'+ '\nSatellite `pcm.py -file $GNSS_ANT_OFF_PCM -epoch $GLOBAL_EPOCH -sat $GNSSLIST -param Antenna1`'],
                    #AmbRes to off
                    ['Global:AmbRes','Off'],
                    ['Global:Input:TimeDepParms:NameFilter:Satellite\.[C]\w*.*\.Clk\.Bias:Degree','0'],
                    ['Global:Input:TimeDepParms:NameFilter:Satellite\.[C]\w*.*\.Clk\.Bias:MaxFormalError','0.4'],
                    ['Global:Input:TimeDepParms:NameFilter:Satellite\.[C]\w*.*\.Clk\.Bias:Strict','On'],
                    ['Global:Input:TimeDepParms:NameFilter:Satellite\.[C]\w*.*\.Clk\.Bias:Strict:MaxDx','1.0e-6'],
                    # Section with namefilter for state
                    ['Global:Input:TimeDepParms:NameFilter:Station\..*\.State\.Pos\..*',' '],
                    ['Global:Input:TimeDepParms:NameFilter:Station\..*\.Trop.*',' '],

                    ['Global:DataTypes:IonoFreePhaseC1C5:DataLinkSpec_LC_BDS:PostSmoothEdit','2e5 2e4 0.25 0.2 0.1 .05'],    
                    ['Global:DataTypes:IonoFreePhaseC1C5:DataLinkSpec_LC_BDS:SignalPath:Platforms','.* C.*'],
    
                    ['Global:DataTypes:IonoFreePhaseP1P2:DataLinkSpec_LC_GPS:PostSmoothEdit','2e5 2e4 0.125 0.1 0.05 .025'],
    
                    ['Global:DataTypes:IonoFreeRangeC1C5:DataLinkSpec_PC_BDS:PostSmoothEdit','2e5 2e4 25 20 10 5'],    
                    ['Global:DataTypes:IonoFreeRangeC1C5:DataLinkSpec_PC_BDS:SignalPath:Platforms','.* C.*'],     

                    ['Global:DataTypes:IonoFreeRangeP1P2:DataLinkSpec_PC_GPS:PostSmoothEdit','2e5 2e4 12.5 10 5 2.5'],     
                ]


In [3]:
from gxlib import gx_aux,gx_convert, gx_merge

In [4]:
import glob
import pandas as pd
import numpy as np
from shutil import rmtree
from multiprocessing import Pool
import os, sys, re, subprocess, calendar

PYGCOREPATH="{}/lib/python{}.{}".format(os.environ['GCOREBUILD'],
                            sys.version_info[0], sys.version_info[1])
if PYGCOREPATH not in sys.path:
    sys.path.insert(0,PYGCOREPATH)

import gipsyx.tropNom as tropNom
import gcore.StationDataBase as StationDataBase
import gcore.treeUtils as treeUtils


class rnx2dr:
    def __init__(self,
                 project_name,
                 stations_list,
                 years_list,
                 rnx_dir='/mnt/Data/bogdanm/GNSS_data/BIGF_data/daily30s',
                 tmp_dir='/mnt/Data/bogdanm/tmp_GipsyX',
                 VMF1_dir = '/mnt/Data/bogdanm/Products/VMF1_Products',
                 tropNom_type = '30h_tropNominalOut_VMF1.tdp',
                 IGS_logs_dir = '/mnt/Data/bogdanm/GNSS_data/BIGF_data/station_log_files',
                 tree_options=tree_options,
                 rate = 300,
                 gnss_products_dir = '/mnt/Data/bogdanm/Products/JPL_GPS_Products/Final',
                 ionex_type='igs', #No ionex dir required as ionex merged products will be put into tmp directory by ionex class
                 num_cores = 8):
        
        self.project_name = project_name
        self.IGS_logs_dir = IGS_logs_dir
        self.rnx_dir=rnx_dir
        self.tmp_dir=tmp_dir
        self.stations_list=stations_list
        self.years_list=years_list
        self.num_cores = num_cores
        self.VMF1_dir = VMF1_dir
        self.tropNom_type = tropNom_type
        self.tree_options = tree_options
        self.rnx_files = gx_convert.select_rnx(rnx_dir=self.rnx_dir,stations_list=self.stations_list,years_list=self.years_list)
        self.rnx_files_in_out = gx_convert.rnx2dr_gen_paths(rnx_files=self.rnx_files,stations_list=self.stations_list,tmp_dir=self.tmp_dir)
        self.staDb_dir= gx_aux.gen_staDb(self.tmp_dir,self.project_name,self.stations_list,self.IGS_logs_dir)
        self.gnss_products_dir = gnss_products_dir
        self.ionex_type=ionex_type
        self.rate=rate
        
    def analyse(self):
        return gx_aux.analyse(rnx_files=self.rnx_files,stations_list=self.stations_list,years_list=self.years_list)
    def rnx2dr(self):
        gx_convert.rnx2dr(rnx_files=self.rnx_files, stations_list=self.stations_list, tmp_dir=self.tmp_dir, num_cores=self.num_cores)

    def get_drInfo(self):
        gx_aux.get_drinfo(num_cores=self.num_cores,rnx_files_in_out=self.rnx_files_in_out,stations_list=self.stations_list,tmp_dir=self.tmp_dir,years_list=self.years_list)
    
    def dr_merge(self):
        merge_table = gx_merge.get_merge_table(tmp_dir=self.tmp_dir)
        gx_merge.dr_merge(merge_table=merge_table,num_cores=self.num_cores,stations_list=self.stations_list)

#     def tdpInput(self,tropnom_param):
#         staDb=StationDataBase.StationDataBase() #creating staDb object
#         staDb.read(self.staDb_dir) #reading staDb into staDb object
#         stns = staDb.getStationList() #creating array with available station names

#         begin,end,tropNom_out = tropnom_param
#         '''begin, end, station_name, tropNom_out
#         begin = tropnom begin in J2000 seconds; end = tropnom end in J2000 seconds; station name as in staDb;  '''
#         if not os.path.isfile(tropNom_out):
#             if not os.path.exists(os.path.dirname(tropNom_out)):
#                 os.makedirs(os.path.dirname(tropNom_out))

#             #begin, end, tdp_PATH
#             nominals=tropNom.nominalTrops('VMF1', modelFile=self.VMF1_dir)
#             nominals.makeTdp(begin, end, self.rate, stns, tropNom_out, append=False, staDb=staDb, dry=True, wet=True)
#     def gen_tropnom(self):
#         '''
#         Generating tropnominal file for valid stations in staDb file. Cannot make it work with VMF.
#         file 31 gives error, no matter what year it is. tdp file is created for each observation file
#         '''
#         num_cores = int(self.num_cores)
        
#         drinfo_file = np.load(file=self.tmp_dir+'/rnx_dr/drinfo.npz')
#         drinfo_years_list = drinfo_file['years_list']

#         #creating folder and file structure taking into account leap year.
#         #resulting paths look as follows: year/doy/30h_tropNominal.vmf1
#         #data on next day needed to create current day tropnominal
#         days_in_year=np.ndarray((len(drinfo_years_list)),dtype=int)
#         for i in range(len(drinfo_years_list)):
            
#             days_in_year[i] = int(365 + (1*calendar.isleap(drinfo_years_list[i])))
#             date = (np.datetime64(str(drinfo_years_list[i])) + (np.arange(days_in_year[i]).astype('timedelta64[D]')))-  self.J2000origin
#             #Now all works correctly. The bug with wrong timevalues was corrected.
#             begin = (date - np.timedelta64(3,'[h]')).astype(int) 
#             end = (date + np.timedelta64(27,'[h]')).astype(int) 

#             paths = (self.tmp_dir +'/tropNom/'+ str(drinfo_years_list[i])+'/'+pd.Series(np.arange(1,days_in_year[i]+1)).astype(str).str.zfill(3)+'/30h_tropNominalOut_VMF1.tdp').values
#             tropnom_param = np.column_stack((begin,end,paths))
            
                
            
#             num_cores = num_cores if len(tropnom_param) > num_cores else len(tropnom_param)
#             step_size = int(np.ceil(len(tropnom_param) / num_cores))

#             print(drinfo_years_list[i],'year tropnominals generation...')
#             print ('Number of files to process:', len(tropnom_param),'| Adj. num_cores:', num_cores,end=' ')

#             for i in range(step_size):
#                 try:
#                     pool = Pool(num_cores)
#                     pool.map(self.tdpInput, tropnom_param[np.arange(i, len(tropnom_param), step_size)])
#                 finally:
#                     pool.close()
#                     pool.join()
#             print('| Done!')
# #         return tropnom_param
            
#     def gen_trees(self):
#         # reading ionex filenames
#         out_df = pd.DataFrame()
# #         default_tree = '/home/bogdanm/Desktop/GipsyX_trees/Trees_kinematic_VMF1_IONEX/ppp_0.tree'
#         default_tree = '/apps/gipsyx/beta/GipsyX-Beta/share/gd2e/DefaultTreeSeries/PPP/ppp_0.tree'
#         input_tree = treeUtils.tree(default_tree)
#         ionex_files = pd.Series(
#             sorted(glob.glob(self.tmp_dir+'/IONEX_merged/' + self.ionex_type + '*')))
#         ionex_basenames = ionex_files.str.split('/', expand=True).iloc[:, -1]

#         out_df['year'] = ionex_basenames.str.slice(3, 7)
#         out_df['tree_path'] = self.tmp_dir + '/Trees/' + \
#             ionex_basenames + '/'  # where to save tree file

#         for i in range(len(ionex_files)):
#             if not os.path.exists(out_df['tree_path'].iloc[i]):
#                 os.makedirs(out_df['tree_path'].iloc[i])
#             input_tree.entries['Global:Ion2nd:StecModel:IonexFile'] = treeUtils.treevalue(
#                 ionex_files[i])
#             input_tree.entries.pop(
#                 'GRN_STATION_CLK_WHITE:State:Pos:ConstantAdj', None)

#             for option in self.tree_options:
#                 input_tree.entries[option[0]] = treeUtils.treevalue(
#                     option[1])  # write standard parameters

#             input_tree.save(out_df['tree_path'][i] + 'ppp_0.tree')
#         # return year type path_trees

#         return out_df.set_index(['year'])   
#     def _gd2e(self, gd2e_set):
        
#         if not os.path.exists(gd2e_set['output']):os.makedirs(gd2e_set['output'])
#         process = subprocess.Popen(['gd2e.py',
#                                     '-drEditedFile', gd2e_set['filename'],
#                                     '-recList', gd2e_set['station'],
#                                     '-runType', 'PPP',
#                                     '-GNSSproducts', self.gnss_products_dir,
#                                     '-treeSequenceDir', gd2e_set['tree_path'],
#                                     '-tdpInput', gd2e_set['tdp'],
#                                     '-staDb', self.staDb_dir,
#                                     '-gdCov'], cwd=gd2e_set['output'],stdout=subprocess.PIPE)
#         out, err = process.communicate()
#         print(str(gd2e_set['year'])+'/'+gd2e_set['dayofyear'])
#         #read tdp file (smooth0_0.tdp)
#         tdp, tdp_header = self._get_tdps_pn(gd2e_set['output']+'/smooth0_0.tdp')
#         #read tree file
#         debug_tree_file = gd2e_set['output']+'/debug.tree'
#         debug_tree = pd.read_csv(debug_tree_file,sep='#',header=None,error_bad_lines=True)
        
#         runAgain = 'gd2e.py -drEditedFile {0} -recList {1} -runType PPP -GNSSproducts {2} -treeSequenceDir {3} -tdpInput {4} -staDb {5} -gdCov'.format(
#             gd2e_set['filename'],gd2e_set['station'],self.gnss_products_dir, gd2e_set['tree_path'],gd2e_set['tdp'],self.staDb_dir)
        
#         rtgx_log = pd.read_csv(gd2e_set['output']+'/rtgx_ppp_0.tree.log0_0',sep='\n',header=None).values
#         rtgx_err = pd.read_csv(gd2e_set['output']+'/rtgx_ppp_0.tree.err0_0',sep='\n',header=None).values
#         #Kill folder with all files after reading
#         rmtree(path=gd2e_set['output'])
#         #create directory to store npz extracted data
#         os.makedirs(gd2e_set['output'])
#         np.savez_compressed(file=gd2e_set['output']+'/gipsyx_out',
#                             tdp=tdp,
#                             tdp_header=tdp_header,
#                             debug_tree=debug_tree,
#                             runAgain=runAgain,
#                             rtgx_log=rtgx_log,
#                             rtgx_err=rtgx_err,
#                             out = np.asarray(out),
#                             err = np.asarray(err))
        
#     def gd2e(self):
#         trees_df = self.gen_trees()
#         gd2e_table = np.ndarray((len(self.stations_list)),dtype=object)
#         tmp_get_merge_tables = self.get_merge_table()
#         #loading list of analysed stations from drinfo npz file
#         drinfo_file = np.load(file=self.tmp_dir+'/rnx_dr/drinfo.npz')
#         drinfo_stations_list = drinfo_file['stations_list']
        
        
#         for i in range(len(self.stations_list)):
#             tmp = pd.DataFrame()
                                
#             station_index_in_drinfo = np.where(drinfo_stations_list==self.stations_list[i])[0][0]
#             tmp_merge_table = tmp_get_merge_tables[station_index_in_drinfo]
            
#             filename = pd.Series(tmp_merge_table[:,4])#<============== Here correct for real station name i in drinfo main table
#             filename[tmp_merge_table[:,0]==3] = filename[tmp_merge_table[:,0]==3].str.slice(start=None, stop=-6) + '_30h.dr.gz'
                        
#             tmp['filename'] = filename
#             tmp['class'] = tmp_merge_table[:,0]
#             tmp['year'] = pd.Series(tmp_merge_table[:,1]).dt.year.astype(str)
#             tmp['dayofyear'] = pd.Series(tmp_merge_table[:,1]).dt.dayofyear.astype(str).str.zfill(3)
#             tmp = tmp.join(other=trees_df,on='year')
#             tmp['tdp'] = self.tmp_dir+'/tropNom/' + tmp['year'] + '/' + tmp['dayofyear'] + '/' + self.tropNom_type
#             tmp['output'] = self.tmp_dir+'/gd2e/'+self.project_name +'/'+self.stations_list[i]+'/'+tmp['year']+ '/' + tmp['dayofyear']


#             tmp['station'] = self.stations_list[i]
            
#             tmp['year'] = pd.to_numeric(tmp['year'])
            
#             tmp = tmp[tmp['year'].isin(self.years_list)&tmp['class']!=0] #cleaning unused years (loaded from npz!!!)
            
#             #Check if files exist (from what left):
#             file_exists = np.zeros(tmp.shape[0],dtype=int)
#             for j in range(tmp.shape[0]):
#                 file_exists[j] = os.path.isfile(tmp['output'].iloc[j]+'/gipsyx_out.npz')
#             tmp['file_exists']=file_exists

#             if tmp[tmp['file_exists']==0].shape[0] ==0:
#                 gd2e_table[i] = None
#                 print('Station', self.stations_list[i], 'is already processed')
#             else:
#                 gd2e_table[i] = tmp[tmp['file_exists']==0].to_records()#converting to records in order for mp to work properly as it doesn't work with pandas Dataframe
#                 num_cores = self.num_cores if len(gd2e_table[i]) > self.num_cores else len(gd2e_table[i])

#                 print('\nStaion',self.stations_list[i],'processing starts...')
#                 print('Number of files to be processed:', len(gd2e_table[i]),
#                       '\nAdjusted number of cores:', num_cores)
#                 try:
#                     pool = Pool(processes=num_cores)
#                     pool.map_async(func=self._gd2e,iterable=gd2e_table[i])
#                 finally:
#                     pool.close()
#                     pool.join()
#                 print('Staion',self.stations_list[i],'done')
#         return gd2e_table
    
#     def _get_tdps_pn(self, file):
#         # A working prototype for fast read and extract of tdp data
#         df = pd.read_table(file, sep='\s+', header=None,
#                                usecols=[0, 1, 2, 3, 4], names=['time','nomvalue', 'value', 'sigma', 'type'])
#         df = df.pivot_table(index='time', columns='type')
        
#         # Create output through dictionary concat
#         extracted_data = pd.concat({
#                                     'sigma'  : df['sigma'].iloc[:,np.arange(-11,-1)],
#                                     'nomvalue': df['nomvalue'].iloc[:,np.arange(-11,-1)],
#                                     'value'   : df['value'].iloc[:,np.arange(-11,-1)]
#                                     },axis=1)
        
#         tdp = np.column_stack((df.index.values,extracted_data.values))
#         tdp_header = extracted_data.columns.values
        
        
#         return tdp,tdp_header

In [5]:
'''CAMB test dataset 2002-2006 processing for noise analysis etc'''
stations_list=['CAMB',]
years_list=[2003,2004,2005,2006]
# years_list=[2002,2003]
camb_aux = rnx2dr(stations_list=stations_list,years_list=years_list, project_name='camb_2002-2006_kinematic',num_cores=20,)

In [5]:
camb_aux.rnx2dr()
# camb_aux.get_drInfo()

CAMB station conversion to binary...
Found 1383 RNX files converted.
Nothing to convert. All available rnx files are already converted


In [6]:
camb_aux.dr_merge()

CAMB station binary files merging (class_3 only)...
Number of files to process: 1113 | Adj. num_cores: 20 | Chunksize: 66 | Done!


In [27]:
camb_aux.rnx_files_in_out[0][:,1]

array(['/mnt/Data/bogdanm/tmp_GipsyX/rnx_dr/CAMB/2003/001/camb0010.03d.Z.dr.gz',
       '/mnt/Data/bogdanm/tmp_GipsyX/rnx_dr/CAMB/2003/002/camb0020.03d.Z.dr.gz',
       '/mnt/Data/bogdanm/tmp_GipsyX/rnx_dr/CAMB/2003/003/camb0030.03d.Z.dr.gz',
       ...,
       '/mnt/Data/bogdanm/tmp_GipsyX/rnx_dr/CAMB/2006/363/camb3630.06d.Z.dr.gz',
       '/mnt/Data/bogdanm/tmp_GipsyX/rnx_dr/CAMB/2006/364/camb3640.06d.Z.dr.gz',
       '/mnt/Data/bogdanm/tmp_GipsyX/rnx_dr/CAMB/2006/365/camb3650.06d.Z.dr.gz'],
      dtype=object)

In [28]:
import numpy as _np
import os as _os
if_exists_array = _np.ndarray((len(camb_aux.rnx_files_in_out[0])), dtype=bool)
                              

for j in range(len(if_exists_array)):
    if_exists_array[j] = not _os.path.exists(camb_aux.rnx_files_in_out[0][j,1])
if_exists_array

array([ True,  True,  True, ...,  True,  True,  True])

In [8]:
drinfo = np.load('/mnt/Data/bogdanm/tmp_GipsyX/rnx_dr/drinfo.npz')

In [12]:
drinfo['drinfo']

array([array([['CAMB', 1894, numpy.datetime64('2003-01-01T00:05:00.000000'),
        ..., 26, array(['G'], dtype=object),
        '/mnt/Data/bogdanm/tmp_GipsyX/rnx_dr/CAMB/2003/001/camb0010.03d.Z.dr.gz'],
       ['CAMB', 2106, numpy.datetime64('2003-01-02T00:05:00.000000'),
        ..., 26, array(['G'], dtype=object),
        '/mnt/Data/bogdanm/tmp_GipsyX/rnx_dr/CAMB/2003/002/camb0020.03d.Z.dr.gz'],
       ['CAMB', 2095, numpy.datetime64('2003-01-03T00:05:00.000000'),
        ..., 26, array(['G'], dtype=object),
        '/mnt/Data/bogdanm/tmp_GipsyX/rnx_dr/CAMB/2003/003/camb0030.03d.Z.dr.gz'],
       ...,
       ['CAMB', 2604, numpy.datetime64('2006-12-29T00:00:00.000000'),
        ..., 31, array(['G'], dtype=object),
        '/mnt/Data/bogdanm/tmp_GipsyX/rnx_dr/CAMB/2006/363/camb3630.06d.Z.dr.gz'],
       ['CAMB', 2449, numpy.datetime64('2006-12-30T00:00:00.000000'),
        ..., 31, array(['G'], dtype=object),
        '/mnt/Data/bogdanm/tmp_GipsyX/rnx_dr/CAMB/2006/364/camb3640.06d.Z.

In [6]:
from gxlib import gx_merge

In [17]:
gx_merge.get_merge_table(tmp_dir=camb_aux.tmp_dir)[0][1]

array([3, numpy.datetime64('2003-01-02T00:05:00.000000'),
       numpy.datetime64('2003-01-02T23:55:00.000000'),
       '/mnt/Data/bogdanm/tmp_GipsyX/rnx_dr/CAMB/2003/001/camb0010.03d.Z.dr.gz',
       '/mnt/Data/bogdanm/tmp_GipsyX/rnx_dr/CAMB/2003/002/camb0020.03d.Z.dr.gz',
       '/mnt/Data/bogdanm/tmp_GipsyX/rnx_dr/CAMB/2003/003/camb0030.03d.Z.dr.gz'],
      dtype=object)

In [22]:
set_test = gx_merge.get_merge_table(tmp_dir=camb_aux.tmp_dir)[0][1]

In [23]:
gx_merge._merge(merge_set=set_test)

In [58]:
import os as _os
from gxlib.gx_aux import J2000origin
from subprocess import Popen as _Popen
def _merge(merge_set):
    '''Expects a merge set of 3 files [class,time,file0,file1,file2]. Merges all files into file1_30h. file1 must be a class 3 file
    Constructs 30h arcs with drMerge.py.
    Sample input:
    drMerge.py -i isba0940.15o.dr ohln0940.15o.dr -start 2015-04-04 00:00:00 -end 2015-04-04 04:00:00
    '''
    if not _os.path.isfile((merge_set[4])[:-6]+'_30h.dr.gz'):
        #Computing time boundaries of the merge
        merge_begin = ((merge_set[1].astype('datetime64') - _np.timedelta64( 3,'[h]'))-J2000origin).astype('timedelta64[s]').astype(int).astype(str)
        merge_end =   ((merge_set[1].astype('datetime64') + _np.timedelta64(27,'[h]'))-J2000origin).astype('timedelta64[s]').astype(int).astype(str)
        
        drMerge_proc = _Popen(['drMerge', merge_begin, merge_end,\
                         os.path.basename(merge_set[4])[:-6]+'_30h.dr.gz',\
                         merge_set[3], merge_set[4], merge_set[5] ],\
                         cwd=_os.path.dirname(merge_set[4]))
        drMerge_proc.wait()

#         drMerge_proc = _Popen(['drMerge', merge_begin, merge_end, merge_set[3], merge_set[4], merge_set[5] ], cwd=_os.path.dirname(merge_set[4]))
#         drMerge_proc.wait()
#         print('drMerge', merge_begin, merge_end, merge_set[3], merge_set[4], merge_set[5])


In [59]:
_merge(set_test)

In [ ]:
drMerge 94727100 94835100 /mnt/Data/bogdanm/tmp_GipsyX/rnx_dr/CAMB/2003/001/camb0010.03d.Z.dr.gz /mnt/Data/bogdanm/tmp_GipsyX/rnx_dr/CAMB/2003/002/camb0020.03d.Z.dr.gz /mnt/Data/bogdanm/tmp_GipsyX/rnx_dr/CAMB/2003/003/camb0030.03d.Z.dr.gz

In [40]:
((_np.datetime64('2003-01-02T00:05:00.000000')- _np.timedelta64( 3,'[h]')) - J2000origin.astype('timedelta64[s]')).astype(int).astype(str)

'94727100000000'

In [39]:
(_np.datetime64('2003-01-02T23:55:00.000000') - J2000origin.astype('timedelta64[s]'))#.astype(int).astype(str)

numpy.datetime64('1973-01-02T11:55:00.000000')

In [9]:
# stations_list=['CAMB','NEWL','BRAE','HERT','LOFT','LERW','WEAR','SHEE']
# years_list=[2007,2008,2009,2010,2011,2012]
stations_list=['CAMB','NEWL','BRAE','HERT','LOFT','LERW','WEAR','SHEE']
# stations_list=['CAMB','NEWL']
years_list=[2007,2008,2009,2010]
# years_list=[2007,]
rnx = rnx2dr(stations_list=stations_list,years_list=years_list, project_name='bigf_np',num_cores=20,)
# rnx.analyse()

In [10]:
rnx.rnx2dr()

CAMB station conversion to binary...
Number of files to process: 1126 | Adj. num_cores: 20 | Chunksize: 57 | Done!
NEWL station conversion to binary...
Number of files to process: 1310 | Adj. num_cores: 20 | Chunksize: 66 | Done!
BRAE station conversion to binary...
Number of files to process: 1441 | Adj. num_cores: 20 | Chunksize: 73 | Done!
HERT station conversion to binary...
Number of files to process: 1451 | Adj. num_cores: 20 | Chunksize: 73 | Done!
LOFT station conversion to binary...
Number of files to process: 1443 | Adj. num_cores: 20 | Chunksize: 73 | Done!
LERW station conversion to binary...
Number of files to process: 1087 | Adj. num_cores: 20 | Chunksize: 55 | Done!
WEAR station conversion to binary...
Number of files to process: 1440 | Adj. num_cores: 20 | Chunksize: 72 | Done!
SHEE station conversion to binary...
Number of files to process: 1157 | Adj. num_cores: 20 | Chunksize: 58 | Done!


In [15]:
rnx._select_rnx()

array([array(['/mnt/Data/bogdanm/GNSS_data/BIGF_data/daily30s/2007/001/camb0010.07d.Z',
       '/mnt/Data/bogdanm/GNSS_data/BIGF_data/daily30s/2007/002/camb0020.07d.Z',
       '/mnt/Data/bogdanm/GNSS_data/BIGF_data/daily30s/2007/003/camb0030.07d.Z',
       ...,
       '/mnt/Data/bogdanm/GNSS_data/BIGF_data/daily30s/2010/082/camb0820.10d.Z',
       '/mnt/Data/bogdanm/GNSS_data/BIGF_data/daily30s/2010/083/camb0830.10d.Z',
       '/mnt/Data/bogdanm/GNSS_data/BIGF_data/daily30s/2010/084/camb0840.10d.Z'],
      dtype='<U70'),
       array(['/mnt/Data/bogdanm/GNSS_data/BIGF_data/daily30s/2007/001/newl0010.07d.Z',
       '/mnt/Data/bogdanm/GNSS_data/BIGF_data/daily30s/2007/002/newl0020.07d.Z',
       '/mnt/Data/bogdanm/GNSS_data/BIGF_data/daily30s/2007/003/newl0030.07d.Z',
       ...,
       '/mnt/Data/bogdanm/GNSS_data/BIGF_data/daily30s/2010/363/newl3630.10d.Z',
       '/mnt/Data/bogdanm/GNSS_data/BIGF_data/daily30s/2010/364/newl3640.10d.Z',
       '/mnt/Data/bogdanm/GNSS_data/BIGF_data/dai

In [10]:
rnx.years_list

[2007, 2008, 2009, 2010]

In [11]:
rnx.rnx_dir

'/mnt/Data/bogdanm/GNSS_data/BIGF_data/daily30s'

In [18]:
gx_aux.select_rnx(rnx_dir=rnx.rnx_dir,stations_list=rnx.stations_list,years_list=rnx.years_list)[0]

array(['/mnt/Data/bogdanm/GNSS_data/BIGF_data/daily30s/2007/001/camb0010.07d.Z',
       '/mnt/Data/bogdanm/GNSS_data/BIGF_data/daily30s/2007/002/camb0020.07d.Z',
       '/mnt/Data/bogdanm/GNSS_data/BIGF_data/daily30s/2007/003/camb0030.07d.Z',
       ...,
       '/mnt/Data/bogdanm/GNSS_data/BIGF_data/daily30s/2010/082/camb0820.10d.Z',
       '/mnt/Data/bogdanm/GNSS_data/BIGF_data/daily30s/2010/083/camb0830.10d.Z',
       '/mnt/Data/bogdanm/GNSS_data/BIGF_data/daily30s/2010/084/camb0840.10d.Z'],
      dtype='<U70')

<module 'lib.aux' from '/home/bogdanm/Desktop/GipsyX_Wrapper/lib/aux.py'>

In [25]:
%aimport lib.aux

In [ ]:
rnx._get_tdps_pn(file='/mnt/Data/bogdanm/tmp_GipsyX/gd2e/2007/CAMB/001/smooth0_0.tdp')

In [ ]:
_get_tdps_pn(file='/mnt/Data/bogdanm/tmp_GipsyX/gd2e/2007/CAMB/001/smooth0_0.tdp')

In [ ]:
# rnx.gen_tropnom()

In [ ]:
def _get_tdps_pn(file):
    # A working prototype for fast read and extract of tdp data
        df = pd.read_table(file, sep='\s+', header=None,
                               usecols=[0, 1, 2, 3, 4], names=['time','nomvalue', 'value', 'sigma', 'type'])
        df = df.pivot_table(index='time', columns='type')
        
        # Create output through dictionary concat
        extracted_data = pd.concat({
                                    'sigma '  : df['sigma'].iloc[:,np.arange(-11,-1)],
                                    'nomvalue': df['nomvalue'].iloc[:,np.arange(-11,-1)],
                                    'value'   : df['value'].iloc[:,np.arange(-11,-1)]
                                    },axis=1)
        
        tdp = np.column_stack((df.index.values,extracted_data.values))
        tdp_header = extracted_data.columns.values
        
        
        return tdp,tdp_header

In [ ]:
tdps = _get_tdps_pn(file='/mnt/Data/bogdanm/tmp_GipsyX/gd2e/2007/CAMB/001/smooth0_0.tdp')
tdps[0][0]

In [ ]:
'''FINAL WORKING VERSION. Processing with tropNominal tdp. no synth'''
years_list=[2007,2008,2009,2010,2011,2012]
stations_list=['CAMB',]T
rnx = rnx2dr(stations_list=stations_list,
                   years_list=years_list,
                   project_name='camb_tdp_in',
                   num_cores=25,
                   tropNom_type='30h_tropNominalOut_VMF1.tdp')
rnx.gd2e()

In [ ]:
'''FINAL WORKING VERSION. Processing small dataset with synthetic waveform in tropNominal tdp'''
years_list=[2007,2008,2009,2010,2011,2012]
stations_list=['CAMB',]
rnx_synth = rnx2dr(stations_list=stations_list,
                   years_list=years_list,
                   project_name='camb_synth_penna',
                   num_cores=10,
                   tropNom_type='30h_tropNominalOut_VMF1.tdp_penna')
rnx_synth.gd2e()

In [ ]:
'''All tropnom xyz are 6 mm'''
years_list=[2007,2008,2009,2010,2011,2012]
stations_list=['CAMB',]
rnx_synth = rnx2dr(stations_list=stations_list,
                   years_list=years_list,
                   project_name='camb_synth_penna_6',
                   num_cores=20,
                   tropNom_type='30h_tropNominalOut_VMF1.tdp_penna_6')
rnx_synth.gd2e()

In [ ]:
'''STADB 0 FINAL WORKING VERSION. Processing small dataset with synthetic waveform in tropNominal tdp'''
years_list=[2007,2008,2009,2010,2011,2012]
stations_list=['CAMB',]
rnx_synth = rnx2dr(stations_list=stations_list,
                   years_list=years_list,
                   project_name='camb_staDb_0',
                   num_cores=10,
                   tropNom_type='30h_tropNominalOut_VMF1.tdp_0_staDb_0')
rnx_synth.gd2e()

In [ ]:
'''STADB 1 FINAL WORKING VERSION. Processing small dataset with synthetic waveform in tropNominal tdp'''
years_list=[2007,2008,2009,2010,2011,2012]
stations_list=['CAMB',]
rnx_synth = rnx2dr(stations_list=stations_list,
                   years_list=years_list,
                   project_name='camb_staDb_1',
                   num_cores=10,
                   tropNom_type='30h_tropNominalOut_VMF1.tdp_0_staDb_1')
rnx_synth.gd2e()

In [ ]:
'''TEST TEST TEST'''
years_list=[2007,]
stations_list=['CAMB',]
rnx_synth = rnx2dr(stations_list=stations_list,
                   years_list=years_list,
                   project_name='camb_synth_test',
                   num_cores=20,
                   tropNom_type='30h_tropNominalOut_VMF1.tdp_synth')

In [ ]:
rnx_synth.gd2e()

In [ ]:
'''Processing small dataset with synthetic waveform in tropNominal tdp'''
years_list=[2007,]
stations_list=['CAMB',]
rnx_synth_penna = rnx2dr(stations_list=stations_list,
                   years_list=years_list,
                   project_name='camb_synth_penna',
                   num_cores=10,
                   tropNom_type='30h_tropNominalOut_VMF1.tdp_synth_penna')

In [ ]:
rnx_synth_penna.gd2e()

In [ ]:
'''Processing small dataset with synthetic waveform in tropNominal tdp (penna) and tids off'''
years_list=[2007,2008,2009,2010,2011,2012]
stations_list=['CAMB',]
rnx_synth_penna_tides_off = rnx2dr(stations_list=stations_list,
                   years_list=years_list,
                   project_name='camb_synth_penna_tides_off',
                   num_cores=20,
                   tree_options = tree_options_tides_off,
                   tropNom_type='30h_tropNominalOut_VMF1.tdp_staDb_synth_1')


In [ ]:
rnx_synth_penna_tides_off.gd2e()

In [ ]:
'''Processing small dataset with synthetic waveform in tropNominal tdp (penna) and tides off'''
years_list=[2007,2008,2009,2010,2011,2012]
stations_list=['CAMB',]
rnx_staDb_tides_off = rnx2dr(stations_list=stations_list,
                   years_list=years_list,
                   project_name='camb_staDb_tides_off',
                   num_cores=20, 
                    tree_options = tree_options_tides_off,
                   tropNom_type='30h_tropNominalOut_VMF1.tdp_0_staDb_s1')


In [ ]:
rnx_staDb_tides_off.gd2e()

In [ ]:
# rnx_synth_penna_ambres_off.gen_trees()

In [ ]:
'''Processing small dataset VMF1 tropNominal tdp (no synth) and ambres off'''
years_list=[2007,2008,2009,2010,2011,2012]
stations_list=['CAMB',]
rnx_ambres_off = rnx2dr(stations_list=stations_list,
                   years_list=years_list,
                   project_name='camb_ambres_off',
                   num_cores=20, 
                    tree_options = tree_options_ambres_off,
                   tropNom_type='30h_tropNominalOut_VMF1.tdp')

In [ ]:
rnx_ambres_off.gd2e()

In [ ]:
# '''Processing small dataset with synthetic waveform in tropNominal tdp/ penna diff'''
# years_list=[2007,2008,2009,2010,2011,2012]
# stations_list=['CAMB',]
# rnx_synth_penna_diff = rnx2dr(stations_list=stations_list,
#                    years_list=years_list,
#                    project_name='camb_synth_penna_diff',
#                    num_cores=10,
#                    tropNom_type='30h_tropNominalOut_VMF1.tdp_synth_penna_diff')

In [ ]:
rnx_synth_penna_diff.gd2e()

In [ ]:
'''Processing small dataset with synthetic waveform in tropNominal tdp'''
years_list=[2007,2008,2009,2010,2011,2012]
stations_list=['CAMB',]
rnx_camb = rnx2dr(stations_list=stations_list,
                   years_list=years_list,
                   project_name='camb_tdp_corrected',
                   num_cores=10,
                   tropNom_type='30h_tropNominalOut_VMF1.tdp')

In [ ]:
rnx_camb.gd2e()

In [ ]:
'''Processing small dataset with synthetic waveform in tropNominal tdp. Nominal values = 0 | StaDb values |  '''
years_list=[2007,2008,2009,2010,2011,2012]
stations_list=['CAMB',]
rnx_camb_0_staDb_0 = rnx2dr(stations_list=stations_list,
                   years_list=years_list,
                   project_name='camb_tdp_corrected_0_staDb_0',
                   num_cores=10,
                   tropNom_type='30h_tropNominalOut_VMF1.tdp_0_staDb_0')

In [ ]:
rnx_camb_0_staDb_0.gd2e()

In [ ]:
# Final processing of test dataset with tdp files of: 0 | Synth | 1
# '30h_tropNominalOut_VMF1.tdp_0_synth_1'
years_list=[2007,2008,2009,2010,2011,2012]
stations_list=['CAMB',]
rnx_camb_0_synth_1 = rnx2dr(stations_list=stations_list,
                   years_list=years_list,
                   project_name='camb_tdp_corrected_0_synth_1',
                   num_cores=10,
                   tropNom_type='30h_tropNominalOut_VMF1.tdp_0_synth_1')


In [ ]:
rnx_camb_0_synth_1.gd2e()

In [ ]:
'''PENNA HERE'''
# Final processing of test dataset with tdp files of: 0 | Synth | 1
# '30h_tropNominalOut_VMF1.tdp_0_synth_1'
years_list=[2007,2008,2009,2010,2011,2012]
stations_list=['CAMB',]
rnx_camb_staDb_synth_1 = rnx2dr(stations_list=stations_list,
                   years_list=years_list,
                   project_name='camb_tdp_corrected_staDb_synth_1',
                   num_cores=12,
                   tropNom_type='30h_tropNominalOut_VMF1.tdp_staDb_synth_1')

In [ ]:
rnx_camb_staDb_synth_1.gd2e()

In [ ]:
'''Processing small dataset with synthetic waveform in tropNominal tdp'''
years_list=[2007,2008,2009,2010,2011,2012]
stations_list=['CAMB',]
rnx_camb_0_staDb_s1 = rnx2dr(stations_list=stations_list,
                   years_list=years_list,
                   project_name='camb_tdp_corrected_0_staDb_s1',
                   num_cores=16,
                   tropNom_type='30h_tropNominalOut_VMF1.tdp_0_staDb_s1')

In [ ]:
rnx_camb_0_staDb_s1.gd2e()

In [ ]:
synth = rnx_synth._get_tdps_pn(file='/home/bogdanm/Desktop/CAMB_synthtdp_test/smooth0_0.tdp')

In [ ]:
proper = rnx_synth._get_tdps_pn(file='/home/bogdanm/Desktop/CAMB_test/smooth0_0.tdp')

In [ ]:
synth_df = pd.DataFrame(synth[0][:,1:],index=synth[0][:,0],columns=synth[1])
proper_df = pd.DataFrame(proper[0][:,1:],index=proper[0][:,0],columns=proper[1])

In [ ]:
synth_df.iloc[0,2]
#4878480.428133955

In [ ]:
np.load('/mnt/Data/bogdanm/tmp_GipsyX/gd2e/camb_staDb/CAMB/2007/001/gipsyx_out.npz')['runAgain']

In [ ]:
J2000_origin